# This Notebook is going to be used for the Capstone Project

In [4]:
import pandas as pd
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Extracting and Processing HTML data, then writing it into a Data Frame

In [5]:
#import libraries needed for extracting and processing html data
import urllib.request, urllib.parse,urllib.error
from bs4 import BeautifulSoup

Now the data is pulled from from the wikipedia page in the shared URL

In [6]:
# pulling the html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')

Now the data from the link is parsed to variables we have setup

In [7]:
#Retrieving data from the html
table = soup.find('table',attrs = { 'class' : 'wikitable sortable jquery-tablesorter'})
table_body = soup.find('tbody')
rows = table_body.find_all('tr')

After we create the variables, we setup lists to store the data of each column after processing

In [8]:
#list for each column
postal_code_list = []
borough_list = []
neighbourhood_list = []

In the next step we define a function to clean the data before we process it, the processed data is then iteratively appended to the lists we createed in the above cells

In [9]:
#creating a function to extract the text from the List_op_rows
def text_extractor(ele):
    temp_str = ele.get_text()
    temp_lst = temp_str.replace('\n','')
    return temp_lst
#print(temp_lst)
#printing elements from the table
for row in rows:
    line = row.find_all('td')
    counter_i = 0
    for ele in line:
        if counter_i == 0:
            postal_code_list.append(text_extractor(ele))
        elif counter_i == 1:
            borough_list.append(text_extractor(ele))
        elif counter_i == 2:
            neighbourhood_list.append(text_extractor(ele))
        else:
            pass
        counter_i = counter_i + 1
#printing the results of the processing
print(postal_code_list[0:5])
print(borough_list[0:5])
print(neighbourhood_list[0:5])


['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront']


In the above output section, we see the first 5 rows of the lists we set up for each column in the table. Now we will dump this data into a Pandas DataFrame

In [10]:
dict_df = {'Postal Code': postal_code_list,'Borough':borough_list,'Neighbourhoods':neighbourhood_list}
toronto_df = pd.DataFrame.from_dict(dict_df)

We print the result of the data frame

In [11]:
print(toronto_df.head())
print('---------------------------------------------')
print(toronto_df.info())
print('----------------------------------------------')

  Postal Code           Borough             Neighbourhoods
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
---------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Postal Code     180 non-null    object
 1   Borough         180 non-null    object
 2   Neighbourhoods  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB
None
----------------------------------------------


Copying our Data Frame as a checkpoint in case we need to retrace our steps

In [12]:
checkpoint = toronto_df.copy()

We will now clean the data frame as per the requirements laid down in the assignement:
1. To ignore all the rows where boroughs are Not Assigned
2. If the neighbourhood is not assigned but the borough exists, then we will consider the borough as the neighbourhood

In [13]:
toronto_df.drop(toronto_df.loc[toronto_df['Borough']=='Not assigned'].index,axis = 0,inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhoods
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
toronto_df.reset_index(drop = True, inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
checkpoint_1 = toronto_df.copy()

In [16]:
toronto_df.loc[toronto_df['Neighbourhoods']=='Not Assigned']

,Postal Code,Borough,Neighbourhoods


There are no Neighbourhoods where the data is 'Not assigned', therefore as requested we obtain the shape of the data frame.

In [17]:
toronto_df.shape

(103, 3)

## Obtaining the Latitude and Longitude for each Neighbourhood.

In [17]:
!pip install geocoder

In [16]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = 'M4A'
# loop until you get the coordinates
#while(lat_lng_coords is None):
g = geocoder.google("{},Toronto".format(postal_code))
lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

TypeError: 'NoneType' object is not subscriptable

In [44]:
print(latitude,longitude)

49.4842897 8.46700036809857


Due to failure from repeated attempts at calling the Geocoder API, we will npw use the file provided in the link http://cocl.us/Geospatial_data

In [22]:
latlng = pd.read_csv('http://cocl.us/Geospatial_data')
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We copy the latlng df to a file_df incase we need to retrace our steps

In [23]:
file_df = latlng.copy()

Now we merge the Toronto Data Frame and the Latitde and Longititude DF

In [24]:
Toronto = toronto_df.merge(file_df, on = 'Postal Code')
Toronto.head()

,Postal Code,Borough,Neighbourhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Mapping the different Postal Codes and Boroughs in Toronto

We import libraries and Dependencies for this task.

In [6]:
# library to handle JSON files
import json 

#!conda install -c conda-forge geopy --yes
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

 # map rendering library
import folium

print('Libraries imported.')

Libraries imported.


**Using the Geopy Library we obtain the Latitude and Longitude of Toronto City**

In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


Now we create a map of Toronto City and add our Boroughs and Postal Codes over it.

In [26]:
# map of Toronto
map_toronto = folium.Map(location = [latitude,longitude], zoom_start = 11)

# add markers to map
for lat, lng, post_code, borough in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Postal Code'], Toronto['Borough']):
    label = '{}, {}'.format(borough, post_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


## A bit of History about Toronto 

**Searching a bit about Toronto from Wikipedia, to  better understand the different Boroughs**

>When Europeans first arrived at the site of present-day Toronto, the vicinity was inhabited by the Iroquois, who had displaced the Wyandot (Huron) people, occupants of the region for centuries before c. 1500. The name Toronto is likely derived from the Iroquoian word tkaronto, meaning "place where trees stand in the water"
>
>On January 1, 1998, Toronto was greatly enlarged, not through traditional annexations, but as an amalgamation of the Municipality of Metropolitan Toronto and its six lower-tier constituent municipalities: East York, Etobicoke, North York, Scarborough, York, and the original city itself. They were dissolved by an act of the Government of Ontario, and formed into a single-tier City of Toronto (colloquially dubbed the "megacity") replacing all six governments.
>
>Toronto encompasses a geographical area formerly administered by many separate municipalities. These municipalities have each developed a distinct history and identity over the years, and their names remain in common use among Torontonians. Former municipalities include East York, Etobicoke, Forest Hill, Mimico, North York, Parkdale, Scarborough, Swansea, Weston and York. Throughout the city there exist hundreds of small neighbourhoods and some larger neighbourhoods covering a few square kilometres.
>
>The inner suburbs are contained within the former municipalities of York and East York. These are mature and traditionally working-class areas, consisting primarily of post–World War I small, single-family homes and small apartment blocks.
>
>The outer suburbs comprising the former municipalities of Etobicoke (west), Scarborough (east) and North York (north) largely retain the grid plan laid before post-war development.Over the late 20th century and early 21st century, North York City Centre, Etobicoke City Centre and Scarborough City Centre have emerged as secondary business districts outside Downtown Toronto. High-rise development in these areas has given the former municipalities distinguishable skylines of their own with high-density transit corridors serving them.

From the above the information we find that the boroughs of Etobicoke, Scarborough and North York were founded later than the Old Toronto City and York borough. We also observe that these new boroughs have made significant strides in urbanisation.

We will therefore attempt to form clusters of the different postal codes in these boroughs, to identify their how different each borough is from each other.

__We make a new dataframe and a new map as required for the analysis__

In [27]:
Toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [28]:
etobicoke_df = Toronto[Toronto['Borough']=='Etobicoke'].reset_index(drop=True)
scarborough_df = Toronto[Toronto['Borough']=='Scarborough'].reset_index(drop=True)
northyork_df = Toronto[Toronto['Borough']=='North York'].reset_index(drop=True)

outer_tn = pd.concat([etobicoke_df,scarborough_df,northyork_df],ignore_index = True)
outer_tn

,Postal Code,Borough,Neighbourhoods,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [110]:
outer_tn.to_csv('Outer_TN.CSV')

In [19]:
outer_tn = pd.read_csv('Outer_TN.CSV')
outer_tn.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhoods,Latitude,Longitude
0,0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [20]:
outer_tn.drop(columns = 'Unnamed: 0',inplace = True)
outer_tn.head()

,Postal Code,Borough,Neighbourhoods,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


We plot the postal codes of the outer suburbs of Toronto only.

In [29]:
# map of Toronto Outer
map_toronto_outer = folium.Map(location = [latitude,longitude], zoom_start = 11)

# add markers to map
for lat, lng, post_code, borough in zip(outer_tn['Latitude'], outer_tn['Longitude'], outer_tn['Postal Code'], outer_tn['Borough']):
    label = '{}, {}'.format(borough, post_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_outer)  
    
map_toronto_outer


## __Now we will use the FourSquare API to get the venues from Outer Toronto__

We define the FourSquare Version

In [81]:
#opening the file with the credentials
with open('FourSquareApi.txt','r') as cred:
    creds = []
    for line in cred:
        lines = line.rstrip()
        creds.append(lines)
    

CLIENT_ID = creds[0]
CLIENT_SECRET = creds[1]
VERSION = '20200701'

Exploring one Postal Code from the Data Frame

In [40]:
# Taking the first row from the outer_tn data frame
neighbourhood_latitude = outer_tn['Latitude'][0]
neighbourhood_longitude= outer_tn['Longitude'][0]
print('The Coordinates for the first postal code {} \n'.format(outer_tn['Postal Code'][0]),' From the borough in the list is {}'.format(outer_tn['Borough'][0]))
print(neighbourhood_latitude)
print(neighbourhood_longitude)

The Coordinates for the first postal code M9A 
  From the borough in the list is Etobicoke
43.6678556
-79.53224240000002


Setting up the URL

In [93]:
# type your answer here
LIMIT = 100

radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

Sending the get requests and checking the results

In [94]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f1e484b8a8c242443e2c031'},
 'response': {'headerLocation': 'Edenbridge - Humber Valley',
  'headerFullLocation': 'Edenbridge - Humber Valley, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.676855609000015,
    'lng': -79.51982358836784},
   'sw': {'lat': 43.65885559099999, 'lng': -79.54466121163219}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bfd53764cf820a13849ecf4',
       'name': "Java Joe's Village Cafe",
       'location': {'address': '1500 Islington Ave',
        'crossStreet': 'at Rathburn Rd',
        'lat': 43.662460906352436,
        'lng': -79.53205381416235,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.662460906352436,
   

Creating a function to extract the category names

In [95]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Creating a data frame for the venues

In [97]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

E:\PythonStuff\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Java Joe's Village Cafe,Café,43.662461,-79.532054
1,St Georges Golf and Country Club,Golf Course,43.674395,-79.537142
2,COBS Bread,Bakery,43.664940,-79.520485
3,TD Canada Trust,Bank,43.662545,-79.531749
4,Shoppers Drug Mart,Pharmacy,43.663067,-79.531753


To understand how many venues were returned by the API

In [98]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


Creating a function to extract all the venues from all the postal codes in the outer tn data frame

In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We call the function

In [100]:

Outer_TN_venues = getNearbyVenues(names=outer_tn['Postal Code'],
                                   latitudes=outer_tn['Latitude'],
                                   longitudes=outer_tn['Longitude']
                                  )


M9A
M9B
M9C
M9P
M9R
M8V
M9V
M8W
M9W
M8X
M8Y
M8Z
M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M3A
M4A
M6A
M3B
M6B
M3C
M2H
M3H
M2J
M3J
M2K
M3K
M2L
M3L
M6L
M9L
M2M
M3M
M5M
M9M
M2N
M3N
M2P
M2R


Checking the result of the data fata frame

In [102]:
Outer_TN_venues.shape

(1275, 7)

In [103]:
Outer_TN_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M9A,43.667856,-79.532242,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,M9A,43.667856,-79.532242,St Georges Golf and Country Club,43.674395,-79.537142,Golf Course
2,M9A,43.667856,-79.532242,COBS Bread,43.664940,-79.520485,Bakery
3,M9A,43.667856,-79.532242,TD Canada Trust,43.662545,-79.531749,Bank
4,M9A,43.667856,-79.532242,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy


Renaming the Neighbourhood column to Postal Code

In [106]:
Outer_TN_venues = Outer_TN_venues.rename(columns = {'Neighborhood':'Postal Code','Neighborhood Latitude':'Latitude','Neighborhood Longitude':'Longitude'})
Outer_TN_venues.head()

,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M9A,43.667856,-79.532242,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,M9A,43.667856,-79.532242,St Georges Golf and Country Club,43.674395,-79.537142,Golf Course
2,M9A,43.667856,-79.532242,COBS Bread,43.664940,-79.520485,Bakery
3,M9A,43.667856,-79.532242,TD Canada Trust,43.662545,-79.531749,Bank
4,M9A,43.667856,-79.532242,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy


creating a checkpoint in case we need to restrace our steps

In [107]:
checkpoint3 = Outer_TN_venues.copy()
checkpoint3.to_csv('Outer_TN_Venues.CSV')

Checking to see how many venues were returned for each neigh bourhood

In [108]:
Outer_TN_venues.groupby('Postal Code').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,21,21,21,21,21,21
M1C,5,5,5,5,5,5
M1E,25,25,25,25,25,25
M1G,9,9,9,9,9,9
M1H,29,29,29,29,29,29
M1J,12,12,12,12,12,12
M1K,29,29,29,29,29,29
M1L,29,29,29,29,29,29
M1M,12,12,12,12,12,12


Number of Unique Categories are

In [109]:
print('There are {} uniques categories.'.format(len(Outer_TN_venues['Venue Category'].unique())))

There are 205 uniques categories.


In [5]:
Outer_TN_venues = pd.read_csv('Outer_TN_Venues.csv')
Outer_TN_venues.head()

,Unnamed: 0,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,M9A,43.667856,-79.532242,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,1,M9A,43.667856,-79.532242,St Georges Golf and Country Club,43.674395,-79.537142,Golf Course
2,2,M9A,43.667856,-79.532242,COBS Bread,43.664940,-79.520485,Bakery
3,3,M9A,43.667856,-79.532242,TD Canada Trust,43.662545,-79.531749,Bank
4,4,M9A,43.667856,-79.532242,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy


In [9]:
Outer_TN_venues = Outer_TN_venues.reset_index(drop=True)
Outer_TN_venues.drop(columns = 'Unnamed: 0',inplace = True)
Outer_TN_venues.head()

,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M9A,43.667856,-79.532242,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,M9A,43.667856,-79.532242,St Georges Golf and Country Club,43.674395,-79.537142,Golf Course
2,M9A,43.667856,-79.532242,COBS Bread,43.664940,-79.520485,Bakery
3,M9A,43.667856,-79.532242,TD Canada Trust,43.662545,-79.531749,Bank
4,M9A,43.667856,-79.532242,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy


## Analysing each neighbourhood

In [10]:
# one hot encoding
tn_onehot = pd.get_dummies(Outer_TN_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tn_onehot['Postal Code'] = Outer_TN_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [tn_onehot.columns[-1]] + list(tn_onehot.columns[:-1])
tn_onehot = tn_onehot[fixed_columns]

tn_onehot.head()

,Postal Code,Accessories Store,African Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,M9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Analysing the size of the new data frame

In [11]:
tn_onehot.shape

(1275, 206)

We try to understand the mean of the frequency of the occurence of each type category

In [12]:
tn_grouped = tn_onehot.groupby('Postal Code').mean().reset_index()
tn_grouped.head()

,Postal Code,Accessories Store,African Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.047619,0.0,0.0,0.0,0.047619,0.0,0.000000,0.0,...,0.0,0.095238,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,M1C,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,M1E,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,M1G,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,M1H,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.034483,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.034483


In [13]:
tn_grouped.shape

(52, 206)

**Printing the top 5 venues by location**

In [14]:
num_top_venues = 5

for Location in tn_grouped['Postal Code']:
    print("----"+Location+"----")
    temp = tn_grouped[tn_grouped['Postal Code'] == Location].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0           Coffee Shop  0.10
1                 Trail  0.10
2  Fast Food Restaurant  0.10
3           Bus Station  0.05
4                Bakery  0.05


----M1C----
                venue  freq
0      Breakfast Spot   0.2
1  Italian Restaurant   0.2
2          Playground   0.2
3                Park   0.2
4        Burger Joint   0.2


----M1E----
                  venue  freq
0           Pizza Place  0.20
1                  Bank  0.08
2           Coffee Shop  0.08
3  Fast Food Restaurant  0.08
4        Discount Store  0.04


----M1G----
                  venue  freq
0           Coffee Shop  0.22
1                  Park  0.22
2     Indian Restaurant  0.11
3              Pharmacy  0.11
4  Fast Food Restaurant  0.11


----M1H----
               venue  freq
0        Coffee Shop  0.10
1             Bakery  0.10
2        Gas Station  0.07
3               Bank  0.07
4  Indian Restaurant  0.07


----M1J----
                  venue  freq
0        Ice Cream

Now we put this data into a pandas data frame

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new data frame to sort the top 10 venues in descending order

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tn_venues_sorted = pd.DataFrame(columns=columns)
tn_venues_sorted['Postal Code'] = tn_grouped['Postal Code']

for ind in np.arange(tn_grouped.shape[0]):
    tn_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tn_grouped.iloc[ind, :], num_top_venues)

tn_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Trail,Coffee Shop,Bank,Greek Restaurant,Martial Arts Dojo,Paper / Office Supplies Store,Sandwich Place,Spa,Caribbean Restaurant
1,M1C,Playground,Breakfast Spot,Park,Italian Restaurant,Burger Joint,Yoga Studio,Event Space,Flower Shop,Flea Market,Fish Market
2,M1E,Pizza Place,Coffee Shop,Fast Food Restaurant,Bank,Restaurant,Discount Store,Smoothie Shop,Food & Drink Shop,Sandwich Place,Sports Bar
3,M1G,Park,Coffee Shop,Chinese Restaurant,Pharmacy,Indian Restaurant,Mobile Phone Shop,Fast Food Restaurant,Eastern European Restaurant,Fish Market,Fish & Chips Shop
4,M1H,Coffee Shop,Bakery,Bank,Indian Restaurant,Gas Station,Music Store,Fast Food Restaurant,Bus Line,Burger Joint,Caribbean Restaurant


## Now we Cluster the Neighbourhood using k Means Clustering algorithm

In [103]:
# set number of clusters
kclusters = 5

tn_grouped_clustering = tn_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0])

In [104]:
# add clustering labels
tn_venues_sorted.drop('Cluster Labels',axis = 1, inplace = True)
tn_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Outer_TN_merged = outer_tn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Outer_TN_merged = Outer_TN_merged.join(tn_venues_sorted.set_index('Postal Code'), on='Postal Code')

Outer_TN_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,0.0,Pharmacy,Convenience Store,Grocery Store,Playground,Skating Rink,Shopping Mall,Café,Bank,Golf Course,Bakery
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,0.0,Park,Pizza Place,Gym,Bank,Clothing Store,Mexican Restaurant,Restaurant,Grocery Store,Café,Convenience Store
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,0.0,Coffee Shop,Convenience Store,Gas Station,Pet Store,College Rec Center,Café,Shopping Mall,Shopping Plaza,Farmers Market,Fish & Chips Shop
3,M9P,Etobicoke,Westmount,43.696319,-79.532242,0.0,Pizza Place,Gas Station,Discount Store,Middle Eastern Restaurant,Breakfast Spot,Supermarket,Sandwich Place,Chinese Restaurant,Coffee Shop,Golf Course
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,0.0,Pharmacy,Intersection,Shopping Mall,Sandwich Place,Beer Store,Gas Station,Supermarket,Supplement Shop,Chinese Restaurant,Coffee Shop


In [105]:
Outer_TN_merged['Cluster Labels'].unique()

array([ 0.,  2.,  3., nan,  1.,  4.])

In [106]:
print(Outer_TN_merged[Outer_TN_merged['Cluster Labels'].isnull()])

   Postal Code      Borough Neighbourhoods   Latitude  Longitude  \
28         M1X  Scarborough    Upper Rouge  43.836125 -79.205636   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
28             NaN                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
28                   NaN                   NaN                   NaN   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
28                   NaN                   NaN                   NaN   

   9th Most Common Venue 10th Most Common Venue  
28                   NaN                    NaN  


Row 28 returnend an NAN value, this is because there are no locations around. This was confirmed after checking its location in the map therefore we drop row 28. And re do the clustering

In [140]:
tn_grouped_52 = tn_grouped.drop(tn_grouped[tn_grouped['Postal Code'] == 'M1X'].index).reset_index()
tn_venues_sorted_52 = tn_venues_sorted.drop(tn_venues_sorted[tn_venues_sorted['Postal Code']=='M1x'].index).reset_index()
outer_tn_52 = outer_tn.drop(outer_tn[outer_tn['Postal Code']=='M1X'].index).reset_index()

In [141]:
# set number of clusters
kclusters = 5

tn_grouped_clustering = tn_grouped_52.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
tn_venues_sorted.drop('Cluster Labels',axis = 1, inplace = True)
tn_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Outer_TN_merged = outer_tn_52

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Outer_TN_merged = Outer_TN_merged.join(tn_venues_sorted.set_index('Postal Code'), on='Postal Code')

Outer_TN_merged.head() # check the last columns!

,index,Postal Code,Borough,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,2,Pharmacy,Convenience Store,Grocery Store,Playground,Skating Rink,Shopping Mall,Café,Bank,Golf Course,Bakery
1,1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,2,Park,Pizza Place,Gym,Bank,Clothing Store,Mexican Restaurant,Restaurant,Grocery Store,Café,Convenience Store
2,2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,2,Coffee Shop,Convenience Store,Gas Station,Pet Store,College Rec Center,Café,Shopping Mall,Shopping Plaza,Farmers Market,Fish & Chips Shop
3,3,M9P,Etobicoke,Westmount,43.696319,-79.532242,2,Pizza Place,Gas Station,Discount Store,Middle Eastern Restaurant,Breakfast Spot,Supermarket,Sandwich Place,Chinese Restaurant,Coffee Shop,Golf Course
4,4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,2,Pharmacy,Intersection,Shopping Mall,Sandwich Place,Beer Store,Gas Station,Supermarket,Supplement Shop,Chinese Restaurant,Coffee Shop


In [142]:
print(Outer_TN_merged['Cluster Labels'].value_counts())
print(Outer_TN_merged[Outer_TN_merged['Cluster Labels'].isnull()])


4    11
0    11
3    10
2    10
1    10
Name: Cluster Labels, dtype: int64
Empty DataFrame
Columns: [index, Postal Code, Borough, Neighbourhoods, Latitude, Longitude, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []


In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, borough,cluster in zip(Outer_TN_merged['Latitude'], Outer_TN_merged['Longitude'],Outer_TN_merged['Postal Code'], Outer_TN_merged['Borough'],Outer_TN_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+' ' +str(borough) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [144]:
# map of Toronto Outer
map_toronto_outer = folium.Map(location = [latitude,longitude], zoom_start = 11)

# add markers to map
for lat, lng, post_code, borough in zip(outer_tn['Latitude'], outer_tn['Longitude'], outer_tn['Postal Code'], outer_tn['Borough']):
    label = '{}, {}'.format(borough, post_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_outer)  
    
map_toronto_outer

### Examining each Cluster

#### Cluster 1

In [148]:
Outer_TN_merged.loc[Outer_TN_merged['Cluster Labels'] == 0, Outer_TN_merged.columns[[1] + list(range(6, Outer_TN_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M8V,0,Park,Pizza Place,Italian Restaurant,Fast Food Restaurant,Pharmacy,Café,Liquor Store,Fried Chicken Joint,Restaurant,Mexican Restaurant
7,M8W,0,Discount Store,Pizza Place,Convenience Store,Gas Station,Liquor Store,Moroccan Restaurant,Sandwich Place,Garden Center,Skating Rink,Coffee Shop
9,M8X,0,Coffee Shop,Park,Burger Joint,Sushi Restaurant,Pub,Breakfast Spot,French Restaurant,Italian Restaurant,Bank,Pizza Place
10,M8Y,0,Park,Italian Restaurant,Eastern European Restaurant,Ice Cream Shop,Bus Stop,Gym / Fitness Center,Shopping Mall,Yoga Studio,Fish Market,Fish & Chips Shop
29,M4A,0,Coffee Shop,Pizza Place,Sporting Goods Shop,Café,Golf Course,French Restaurant,Grocery Store,Gym / Fitness Center,Men's Store,Portuguese Restaurant
30,M6A,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Dessert Shop,Restaurant,Sushi Restaurant,Fried Chicken Joint,Bowling Alley
32,M6B,0,Grocery Store,Fast Food Restaurant,Italian Restaurant,Pizza Place,Gas Station,Coffee Shop,Bus Line,Shoe Store,Mediterranean Restaurant,Metro Station
42,M6L,0,Coffee Shop,Mediterranean Restaurant,Bakery,Convenience Store,Park,Dim Sum Restaurant,Chinese Restaurant,Gas Station,Pizza Place,Athletics & Sports
45,M3M,0,Vietnamese Restaurant,Baseball Field,Food Truck,Yoga Studio,Event Space,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
46,M5M,0,Coffee Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Restaurant,Hobby Shop,American Restaurant


#### Cluster 2

In [149]:
Outer_TN_merged.loc[Outer_TN_merged['Cluster Labels'] == 1, Outer_TN_merged.columns[[1] + list(range(6, Outer_TN_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M1P,1,Coffee Shop,Restaurant,Pharmacy,Asian Restaurant,Electronics Store,Indian Restaurant,Light Rail Station,Fast Food Restaurant,Chinese Restaurant,Furniture / Home Store
23,M1R,1,Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,Intersection,Gas Station,Furniture / Home Store,Korean Restaurant,Breakfast Spot,Soccer Field
24,M1S,1,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Bakery,Pizza Place,Lounge,Malay Restaurant,Seafood Restaurant,Sandwich Place,Café
25,M1T,1,Coffee Shop,Pharmacy,Sandwich Place,Convenience Store,Bank,Pizza Place,Intersection,Fast Food Restaurant,Market,Italian Restaurant
26,M1V,1,Chinese Restaurant,Pizza Place,Noodle House,Park,Hobby Shop,Japanese Restaurant,Dessert Shop,Coffee Shop,Caribbean Restaurant,Event Space
27,M1W,1,Chinese Restaurant,Bakery,Fast Food Restaurant,Pizza Place,Coffee Shop,Bank,Grocery Store,Nail Salon,Camera Store,Sandwich Place
34,M2H,1,Park,Coffee Shop,Pharmacy,Shopping Mall,Sandwich Place,Chinese Restaurant,Restaurant,Korean Restaurant,Residential Building (Apartment / Condo),Recreation Center
36,M2J,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Sandwich Place,Bank,Bakery,Juice Bar,Restaurant,Japanese Restaurant,Fried Chicken Joint
38,M2K,1,Grocery Store,Bank,Gas Station,Japanese Restaurant,Café,Park,Chinese Restaurant,Shopping Mall,Restaurant,Trail
40,M2L,1,Park,Pool,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fireworks Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 3

In [150]:
Outer_TN_merged.loc[Outer_TN_merged['Cluster Labels'] == 2, Outer_TN_merged.columns[[1] + list(range(6, Outer_TN_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,2,Pharmacy,Convenience Store,Grocery Store,Playground,Skating Rink,Shopping Mall,Café,Bank,Golf Course,Bakery
1,M9B,2,Park,Pizza Place,Gym,Bank,Clothing Store,Mexican Restaurant,Restaurant,Grocery Store,Café,Convenience Store
2,M9C,2,Coffee Shop,Convenience Store,Gas Station,Pet Store,College Rec Center,Café,Shopping Mall,Shopping Plaza,Farmers Market,Fish & Chips Shop
3,M9P,2,Pizza Place,Gas Station,Discount Store,Middle Eastern Restaurant,Breakfast Spot,Supermarket,Sandwich Place,Chinese Restaurant,Coffee Shop,Golf Course
4,M9R,2,Pharmacy,Intersection,Shopping Mall,Sandwich Place,Beer Store,Gas Station,Supermarket,Supplement Shop,Chinese Restaurant,Coffee Shop
6,M9V,2,Pizza Place,Grocery Store,Beer Store,Caribbean Restaurant,Sandwich Place,Bus Line,Fried Chicken Joint,Discount Store,Liquor Store,Fast Food Restaurant
8,M9W,2,Hotel,Rental Car Location,Coffee Shop,Lounge,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fireworks Store
11,M8Z,2,Restaurant,Burrito Place,Gym / Fitness Center,BBQ Joint,Coffee Shop,Pet Store,Bank,Bakery,Sushi Restaurant,Convenience Store
43,M9L,2,Bank,Italian Restaurant,Pizza Place,Pharmacy,Bakery,Park,Electronics Store,Shopping Mall,Event Space,Falafel Restaurant
47,M9M,2,Intersection,Discount Store,Convenience Store,Bakery,Gas Station,Park,Storage Facility,Golf Course,Falafel Restaurant,Farmers Market


#### Cluster 4

In [151]:
Outer_TN_merged.loc[Outer_TN_merged['Cluster Labels'] == 3, Outer_TN_merged.columns[[1] + list(range(6, Outer_TN_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1B,3,Fast Food Restaurant,Trail,Coffee Shop,Bank,Greek Restaurant,Martial Arts Dojo,Paper / Office Supplies Store,Sandwich Place,Spa,Caribbean Restaurant
13,M1C,3,Playground,Breakfast Spot,Park,Italian Restaurant,Burger Joint,Yoga Studio,Event Space,Flower Shop,Flea Market,Fish Market
14,M1E,3,Pizza Place,Coffee Shop,Fast Food Restaurant,Bank,Restaurant,Discount Store,Smoothie Shop,Food & Drink Shop,Sandwich Place,Sports Bar
15,M1G,3,Park,Coffee Shop,Chinese Restaurant,Pharmacy,Indian Restaurant,Mobile Phone Shop,Fast Food Restaurant,Eastern European Restaurant,Fish Market,Fish & Chips Shop
16,M1H,3,Coffee Shop,Bakery,Bank,Indian Restaurant,Gas Station,Music Store,Fast Food Restaurant,Bus Line,Burger Joint,Caribbean Restaurant
17,M1J,3,Ice Cream Shop,Bowling Alley,Grocery Store,Convenience Store,Pizza Place,Fast Food Restaurant,Train Station,Sandwich Place,Coffee Shop,Restaurant
18,M1K,3,Coffee Shop,Discount Store,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Grocery Store,Bank,Department Store,Rental Car Location,Sandwich Place
19,M1L,3,Intersection,Park,Coffee Shop,Convenience Store,Bakery,Bus Line,Diner,Restaurant,Beer Store,Sandwich Place
20,M1M,3,Pizza Place,Beach,Ice Cream Shop,Pharmacy,Hardware Store,Park,Sports Bar,Restaurant,Farmers Market,Electronics Store
21,M1N,3,Gym Pool,Thai Restaurant,General Entertainment,Café,Ice Cream Shop,Park,Skating Rink,College Stadium,Restaurant,Diner


#### Cluster 5

In [152]:
Outer_TN_merged.loc[Outer_TN_merged['Cluster Labels'] == 4, Outer_TN_merged.columns[[1] + list(range(6, Outer_TN_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M3A,4,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Fish & Chips Shop,Café,Food & Drink Shop,Fast Food Restaurant,Shop & Service
31,M3B,4,Japanese Restaurant,Coffee Shop,Burger Joint,Pizza Place,Bank,Café,Caribbean Restaurant,Salad Place,Liquor Store,Breakfast Spot
33,M3C,4,Restaurant,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Sporting Goods Shop,Supermarket,Café,Burger Joint,Dim Sum Restaurant
35,M3H,4,Bank,Park,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Supermarket,Diner,Dog Run,Mobile Phone Shop,Restaurant
37,M3J,4,Pizza Place,Coffee Shop,Furniture / Home Store,Restaurant,Bank,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Falafel Restaurant,Fast Food Restaurant
39,M3K,4,Coffee Shop,Turkish Restaurant,Food Court,Middle Eastern Restaurant,Café,Park,Soccer Field,Sandwich Place,Liquor Store,Latin American Restaurant
41,M3L,4,Park,Spa,Grocery Store,Coffee Shop,Moving Target,Shopping Mall,Bank,Pizza Place,Vietnamese Restaurant,Gym Pool
44,M2M,4,Korean Restaurant,Café,Park,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Diner,Japanese Restaurant,Bus Station,Bus Line
48,M2N,4,Coffee Shop,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Korean Restaurant,Japanese Restaurant,Sandwich Place,Bank,Restaurant,Fast Food Restaurant
50,M2P,4,Park,Coffee Shop,Restaurant,Golf Course,Grocery Store,Playground,French Restaurant,Bowling Alley,Chinese Restaurant,Gas Station


# Results

**Observations**
1. Each Borough has a high concentration of one type of Cluster, with the exception of Scarborough.
2. We observe in Scarborough, there are 2 clusters dividing the borough evenly in half.
3. We observe, the area around inner Toronto belonging to the same cluster. With the exception of M3N in North York.

**Inferences**
1. We can infer from the above observations that, each borough has developed uniquely with their own unque groups of venues/locations
2. We also infer that Scarborough has 2 clusters due to how different the locations are from eachother within the same borough. That the locations near the river are more similar and different from the locations more inland
3. We also infer that the area of Etobicoke which is near the river, has locations belonging to the same cluster as those locations which are on the outskirts of inner Toronto


### Conclusion

The outer boroughs of Toronto have developed uniquely and they have their own set of unique locations as compared to the other Boroughs.